In [3]:
import numpy as np
import torch
from PIL import Image
from numpy import asarray
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob

In [4]:
# Rearranges matrix into a column vector

def im2col(image, block_size = (1,1), skip=1):

    rows, cols = image.shape
    vert_blocks = rows - block_size[0] + 1

    output_vectors = np.zeros((block_size[0] * block_size[1], cols * rows))
    itr = 0
    for row in range(rows):
        for col in range(cols):
            output_vectors[:, itr] = image[row: row + block_size[0], col: col + 1].ravel()
            itr += 1

    return output_vectors[:, ::skip]

# Performs function of Fig2 (left), creates data matrix X, and outputs the covariance of this matrix. 
# Requires input of ONE CHANNEL, not RGB

def one_channel_to_cov(image, filter_size):
    x, y = filter_size
    H, W = image.shape
    X = [0]*(x*y)
    for i in range(x):
        for j in range(y):
            
            X[y*i + j] = im2col(np.array(image[i:W+i-x+1, j:H+j-y+1]), )


    X = np.squeeze(np.asarray(X)).T

    u = np.mean(X)
    N = X.shape[0]

    X = (X-u).T @ X
    Cov = X / N
    
    return Cov

# This function is provided in the Network Deconvolution paper.
# Numerical approximation of inverse sqrt. Used to invertsqrt the Cov matrix to get Deconv matrix D.

def isqrt_newton_schulz_autograd(A, numIters):
    dim = A.shape[0]
    normA=A.norm()
    Y = A.div(normA)
    I = torch.eye(dim,dtype=A.dtype,device=A.device)
    Z = torch.eye(dim,dtype=A.dtype,device=A.device)

    for i in range(numIters):
        T = 0.5*(3.0*I - Z@Y)
        Y = Y@T
        Z = T@Z
    #A_sqrt = Y*torch.sqrt(normA)
    A_isqrt = Z / torch.sqrt(normA)
    print(A_isqrt)
    return A_isqrt

# plot kernel in 3D
def plot_3D(ker):
    # Set up grid and test data
    nx, ny = 15, 15
    x = range(nx)
    y = range(ny)

    hf = plt.figure()
    ha = hf.add_subplot(111, projection='3d')

    X, Y = np.meshgrid(x, y)  
    ha.plot_surface(X, Y, np.reshape(ker, (15,15)))

    plt.show()

################
# Executes above functions to take ONE CHANNEL input, return kernel (as a vector)

def channel_to_kernel(image, k):
  # Data Matrix X
    cov = one_channel_to_cov(image, (k, k))

    # Deconv matrix D is equal to inverse sqrt of cov matrix. 
    D = isqrt_newton_schulz_autograd(torch.Tensor(cov), 10)

    
    # Extract kernel from Deconv matrix
    ker = torch.diagonal(D)

    return ker


In [5]:
#SET FILTER SIZE
k = 15

In [6]:
# im_name must be string ending in .jpeg

def load_image(im_name, rows, cols, channels):
    #load image
    image = Image.open(im_name)
    # convert image to numpy array
    data = asarray(image)

    data_square = data
    #Resize to square, reduce size for computational efficiency
    data_square = np.resize(data_square, (rows, cols, channels))

    
    return data_square

In [10]:
# ImageNet 128 is a local folder containing 128 randomly selected JPEG images from ImageNet
filenames = glob.glob("/Users/elliotstein/Documents/Semester 2/Deep Learning/ImageNet 128/*.JPEG")

In [ ]:
# Create dictionary storing all images
# Each image is formatted by the load_image function, stored under name data1, data2 etc.
d = {}
N = len(filenames)
for x in range(N):
    d["data{0}".format(x)] = load_image(filenames[x], 224, 224, 3)
    
    
RGB = ['Red', 'Green', 'Blue']
for i in range(3):
    # Reset and initialise ker variable for storing kernel
    ker = None
    ker = torch.zeros((k,k))    
    
    for x in range(N):
        print(RGB[i], ': ', x, "/ N")
        
        
        # Takes individual RGB channel from image, creates deconvolutional kernel and adds to ker
        ker+=torch.Tensor(channel_to_kernel(d["data{0}".format(5)][:,:,i], k)).reshape(15,15)
             
        # Stores three variables, one for each of the RGB channels. Each is the average over all image 
        # deconvolutional kernels
    if i == 0:
        ker0 = ker/N
    if i == 1:
        ker1 = ker/N
    if i == 0:
        ker2 = ker/N

Red
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05,  ...,  6.3349e-02,
         -1.8457e-02, -6.8750e-03],
        [-1.2863e-04, -1.8802e-04, -1.5273e-04,  ..., -1.8456e-02,
          6.2402e-02, -2.1909e-02],
        [-2.1499e-04, -1.3160e-04, -1.9699e-04,  ..., -6.8699e-03,
         -2.1905e-02,  5.0403e-02]])
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427

tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05,  ...,  6.3349e-02,
         -1.8457e-02, -6.8750e-03],
        [-1.2863e-04, -1.8802e-04, -1.5273e-04,  ..., -1.8456e-02,
          6.2402e-02, -2.1909e-02],
        [-2.1499e-04, -1.3160e-04, -1.9699e-04,  ..., -6.8699e-03,
         -2.1905e-02,  5.0403e-02]])
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05

tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05,  ...,  6.3349e-02,
         -1.8457e-02, -6.8750e-03],
        [-1.2863e-04, -1.8802e-04, -1.5273e-04,  ..., -1.8456e-02,
          6.2402e-02, -2.1909e-02],
        [-2.1499e-04, -1.3160e-04, -1.9699e-04,  ..., -6.8699e-03,
         -2.1905e-02,  5.0403e-02]])
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05

tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05,  ...,  6.3349e-02,
         -1.8457e-02, -6.8750e-03],
        [-1.2863e-04, -1.8802e-04, -1.5273e-04,  ..., -1.8456e-02,
          6.2402e-02, -2.1909e-02],
        [-2.1499e-04, -1.3160e-04, -1.9699e-04,  ..., -6.8699e-03,
         -2.1905e-02,  5.0403e-02]])
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05

tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05,  ...,  6.3349e-02,
         -1.8457e-02, -6.8750e-03],
        [-1.2863e-04, -1.8802e-04, -1.5273e-04,  ..., -1.8456e-02,
          6.2402e-02, -2.1909e-02],
        [-2.1499e-04, -1.3160e-04, -1.9699e-04,  ..., -6.8699e-03,
         -2.1905e-02,  5.0403e-02]])
tensor([[ 5.3804e-02, -2.1616e-02, -7.1601e-03,  ..., -1.3627e-04,
         -5.4005e-05, -8.0134e-05],
        [-2.1611e-02,  6.4753e-02, -1.8171e-02,  ..., -1.2111e-04,
         -1.3887e-04, -5.0194e-05],
        [-7.1493e-03, -1.8167e-02,  6.5837e-02,  ..., -6.2341e-05,
         -1.1116e-04, -1.3016e-04],
        ...,
        [-1.9259e-04, -1.5980e-04, -9.5427e-05

...

tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06,  ...,  6.3988e-02,
         -1.9557e-02, -5.9647e-03],
        [-1.1539e-05,  6.7994e-05, -6.4163e-05,  ..., -1.9556e-02,
          6.3340e-02, -2.2567e-02],
        [ 9.3646e-05, -6.5640e-06,  5.7040e-05,  ..., -5.9616e-03,
         -2.2565e-02,  5.0519e-02]])
tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06

tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06,  ...,  6.3988e-02,
         -1.9557e-02, -5.9647e-03],
        [-1.1539e-05,  6.7994e-05, -6.4163e-05,  ..., -1.9556e-02,
          6.3340e-02, -2.2567e-02],
        [ 9.3646e-05, -6.5640e-06,  5.7040e-05,  ..., -5.9616e-03,
         -2.2565e-02,  5.0519e-02]])
tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06

tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06,  ...,  6.3988e-02,
         -1.9557e-02, -5.9647e-03],
        [-1.1539e-05,  6.7994e-05, -6.4163e-05,  ..., -1.9556e-02,
          6.3340e-02, -2.2567e-02],
        [ 9.3646e-05, -6.5640e-06,  5.7040e-05,  ..., -5.9616e-03,
         -2.2565e-02,  5.0519e-02]])
tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06

tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06,  ...,  6.3988e-02,
         -1.9557e-02, -5.9647e-03],
        [-1.1539e-05,  6.7994e-05, -6.4163e-05,  ..., -1.9556e-02,
          6.3340e-02, -2.2567e-02],
        [ 9.3646e-05, -6.5640e-06,  5.7040e-05,  ..., -5.9616e-03,
         -2.2565e-02,  5.0519e-02]])
tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06

tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06,  ...,  6.3988e-02,
         -1.9557e-02, -5.9647e-03],
        [-1.1539e-05,  6.7994e-05, -6.4163e-05,  ..., -1.9556e-02,
          6.3340e-02, -2.2567e-02],
        [ 9.3646e-05, -6.5640e-06,  5.7040e-05,  ..., -5.9616e-03,
         -2.2565e-02,  5.0519e-02]])
tensor([[ 5.3756e-02, -2.2524e-02, -6.2803e-03,  ...,  6.5575e-05,
          1.3889e-05,  1.3802e-04],
        [-2.2521e-02,  6.5782e-02, -1.9455e-02,  ..., -5.0769e-05,
          8.5149e-05,  2.0924e-05],
        [-6.2740e-03, -1.9453e-02,  6.6549e-02,  ...,  1.4154e-05,
         -4.8812e-05,  8.1655e-05],
        ...,
        [ 4.4999e-05, -6.4951e-05,  1.5225e-06

In [ ]:
print('Red Channel Deconv Kernel')
plot_3D(np.array(ker0))
print('Green Channel Deconv Kernel')
plot_3D(np.array(ker1))
print('Blue Channel Deconv Kernel')
plot_3D(np.array(ker2))

In [ ]:
# Random ImageNet images downloaded and named im1-im10 in local folder
